In [2]:
import os

# Set the base path to the current working directory
base_path = os.getcwd()

print(f"Models and scalers will be saved in: {base_path}")

Models and scalers will be saved in: C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction


In [13]:
import requests

BINANCE_BASE_URL = "https://api.binance.com/api/v3"

def fetch_available_pairs():
    response = requests.get(f"{BINANCE_BASE_URL}/exchangeInfo")
    symbols = response.json().get("symbols", [])
    return {s["symbol"] for s in symbols if s["status"] == "TRADING"}

def check_combinations(crypto_symbols, base_symbols, time_frames):
    available_pairs = fetch_available_pairs()
    unsupported_combinations = []
    supported = []

    for crypto_symbol, base_symbol in product(crypto_symbols, base_symbols):
        symbol = f"{crypto_symbol}{base_symbol}"
        if symbol not in available_pairs:
            unsupported_combinations.append(symbol)
        else:
            supported.append(symbol)

    # Check time frames if needed
    # Note: This is a placeholder as time frames are generally supported across all pairs
    supported_time_frames = {"1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"}
    for time_frame in time_frames:
        if time_frame not in supported_time_frames:
            print(f"Unsupported time frame: {time_frame}")

    return unsupported_combinations, supported

# Example usage
CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", "LTC", "TRX", "AVAX", "ATOM", "LINK"] 
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]

TIME_FRAMES = ["15m", "30m", "1h", "4h", "1d", "1w"]

unsupported, supported = check_combinations(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES)
print("Unsupported combinations:", unsupported)
print(f"{len(unsupported)}")

print("Supported combinations:", supported)
print(f"{len(supported)}")


Unsupported combinations: ['DOTJPY', 'LTCJPY', 'TRXJPY', 'AVAXJPY', 'ATOMJPY', 'LINKJPY']
6
Supported combinations: ['BTCUSDT', 'BTCUSDC', 'BTCEUR', 'BTCJPY', 'BTCTRY', 'ETHUSDT', 'ETHUSDC', 'ETHEUR', 'ETHJPY', 'ETHTRY', 'BNBUSDT', 'BNBUSDC', 'BNBEUR', 'BNBJPY', 'BNBTRY', 'XRPUSDT', 'XRPUSDC', 'XRPEUR', 'XRPJPY', 'XRPTRY', 'ADAUSDT', 'ADAUSDC', 'ADAEUR', 'ADAJPY', 'ADATRY', 'SOLUSDT', 'SOLUSDC', 'SOLEUR', 'SOLJPY', 'SOLTRY', 'DOGEUSDT', 'DOGEUSDC', 'DOGEEUR', 'DOGEJPY', 'DOGETRY', 'DOTUSDT', 'DOTUSDC', 'DOTEUR', 'DOTTRY', 'SHIBUSDT', 'SHIBUSDC', 'SHIBEUR', 'SHIBJPY', 'SHIBTRY', 'LTCUSDT', 'LTCUSDC', 'LTCEUR', 'LTCTRY', 'TRXUSDT', 'TRXUSDC', 'TRXEUR', 'TRXTRY', 'AVAXUSDT', 'AVAXUSDC', 'AVAXEUR', 'AVAXTRY', 'ATOMUSDT', 'ATOMUSDC', 'ATOMEUR', 'ATOMTRY', 'LINKUSDT', 'LINKUSDC', 'LINKEUR', 'LINKTRY']
64


In [ ]:
import requests

def fetch_top_50_symbols():
    BINANCE_BASE_URL = "https://api.binance.com/api/v3"
    response = requests.get(f"{BINANCE_BASE_URL}/ticker/24hr")
    data = response.json()

    # Sort by volume and get the top 50
    sorted_data = sorted(data, key=lambda x: float(x['quoteVolume']), reverse=True)
    top_50 = sorted_data[:50]

    # Extract symbols
    top_50_symbols = [item['symbol'] for item in top_50]
    return top_50_symbols

top_50_symbols = fetch_top_50_symbols()
print("Top 50 Symbols by Volume:")
for symbol in top_50_symbols:
    print(symbol)



In [16]:
from itertools import product
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import requests
import pickle

# Define your symbols and time frames
# CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "DOT", "MATIC", "SHIB", "LTC", "TRX", "AVAX", "ATOM", "LINK"]
# BASE_SYMBOLS = ["USDT", "BUSD", "USDC", "BTC", "ETH", "BNB", "EUR", "GBP", "AUD", "JPY", "TRY", "RUB", "CAD", "CNY", "KRW"]
# TIME_FRAMES = ["15m", "30m", "1h", "2h", "4h", "1d", "3d", "1w", "1M"]

CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", "LTC", "TRX", "AVAX", "ATOM", "LINK"] 
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]

TIME_FRAMES = ["15m", "30m", "1h", "4h", "1d", "1w"]

# Base path for saving models and scalers
base_path = base_path = os.getcwd()

# Function to fetch candlestick data
def fetch_candlestick_data(symbol, interval, limit=1000):
    BINANCE_BASE_URL = "https://api.binance.com/api/v3"
    url = f"{BINANCE_BASE_URL}/klines?symbol={symbol}&interval={interval}&limit={limit}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data, columns=[
        'OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume',
        'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'])
    return df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

# Function to train the model
def train_model(symbol, interval, base_path):
    model_path = os.path.join(base_path, f"{symbol}_{interval}_model.keras")
    scaler_path = os.path.join(base_path, f"{symbol}_{interval}_scaler.pkl")

    data = fetch_candlestick_data(symbol, interval)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    x_train, y_train = [], []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i-60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 1)))
    model.add(LSTM(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='loss', verbose=1)

    model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping, model_checkpoint])

    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

    return model, scaler

# Iterate over all combinations
for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
    symbol = f"{crypto_symbol}{base_symbol}"
    print(f"Training model for {symbol} at {time_frame} interval")
    
    # Train the model
    try:
        train_model(symbol, time_frame, base_path)
    except Exception as e:
        print(f"Failed to train model for {symbol} at {time_frame}: {e}")


Training model for BTCUSDT at 15m interval
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.1124
Epoch 1: loss improved from inf to 0.04725, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 74ms/step - loss: 0.1103
Epoch 2/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0080
Epoch 2: loss improved from 0.04725 to 0.00641, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 63ms/step - loss: 0.0079
Epoch 3/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - loss: 0.0045
Epoch 3: loss improved from 0.00641 to 0.00439, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 66ms/step - loss: 0.0045
Epoch 4/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0044
Epoch 4: loss improved from 0.00439 to 0.00406, saving model to C:\Use

Epoch 29: early stopping
Training model for BTCUSDT at 30m interval
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.0800
Epoch 1: loss improved from inf to 0.03546, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_30m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - loss: 0.0785
Epoch 2/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0080
Epoch 2: loss improved from 0.03546 to 0.00694, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_30m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 0.0079
Epoch 3/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0052
Epoch 3: loss improved from 0.00694 to 0.00498, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_30m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.0051
Epoch 4/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0052
Epoch 4: loss did not improve from 0.00498
30

29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0038
Epoch 23: loss did not improve from 0.00353
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - loss: 0.0038
Epoch 24/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0034
Epoch 24: loss improved from 0.00353 to 0.00339, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_1h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.0034
Epoch 25/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0034
Epoch 25: loss did not improve from 0.00339
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - loss: 0.0034
Epoch 26/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0035
Epoch 26: loss did not improve from 0.00339
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - loss: 0.0035
Epoch 27/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0034
Epoch 27: loss did not improve from 0.00339
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 0.0034
Epoch 28/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0027
Epoch

Epoch 20/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0016
Epoch 20: loss improved from 0.00200 to 0.00172, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 96ms/step - loss: 0.0016
Epoch 21/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.0015
Epoch 21: loss improved from 0.00172 to 0.00167, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDT_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 0.0015
Epoch 22/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0022
Epoch 22: loss did not improve from 0.00167
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - loss: 0.0022
Epoch 23/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0020
Epoch 23: loss did not improve from 0.00167
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 70ms/step - loss: 0.0020
Epoch 24/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0017
Epoch 24: loss did not improve from 0.00167
30/30

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0042
Epoch 3: loss improved from 0.00505 to 0.00378, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDC_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0042
Epoch 4/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 0.0036
Epoch 4: loss improved from 0.00378 to 0.00367, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCUSDC_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - loss: 0.0036
Epoch 5/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0034
Epoch 5: loss did not improve from 0.00367
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0034
Epoch 6/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 0.0038
Epoch 6: loss did not improve from 0.00367
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 75ms/step - loss: 0.0038
Epoch 7/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0038
Epoch 7: loss improved from 0.00367 to 0.00363, saving model to C:\

...

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - loss: 0.0096
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0087
Epoch 8: loss did not improve from 0.00991
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0089
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0080
Epoch 9: loss improved from 0.00991 to 0.00764, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCEUR_1w_model.keras
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - loss: 0.0080
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0082 
Epoch 10: loss did not improve from 0.00764
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - loss: 0.0086
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0104
Epoch 11: loss did not improve from 0.00764
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0102
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0077
Epoch 12: loss did not improve from 0.00764
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - loss: 0.0080
Epoch 13/50
7/7 ━━━━━━━━━━━━━━━

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 70ms/step - loss: 0.0046
Epoch 39/50
6/7 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0039
Epoch 39: loss did not improve from 0.00349
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - loss: 0.0040
Epoch 40/50
6/7 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0040
Epoch 40: loss did not improve from 0.00349
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - loss: 0.0039
Epoch 41/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0039
Epoch 41: loss did not improve from 0.00349
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0039
Epoch 41: early stopping
Training model for BTCJPY at 15m interval
Epoch 1/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0672
Epoch 1: loss improved from inf to 0.02862, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_15m_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - loss: 0.0659
Epoch 2/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.0050
Epoch 2: loss improved from 0.02862 to 0.00431, saving mod

Epoch 13/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.0034
Epoch 13: loss improved from 0.00360 to 0.00344, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_1h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - loss: 0.0034
Epoch 14/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0034
Epoch 14: loss improved from 0.00344 to 0.00315, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_1h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - loss: 0.0034
Epoch 15/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0032
Epoch 15: loss did not improve from 0.00315
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 88ms/step - loss: 0.0033
Epoch 16/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.0030
Epoch 16: loss did not improve from 0.00315
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - loss: 0.0030
Epoch 17/50
29/30 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0032
Epoch 17: loss did not improve from 0.00315
30/30 ━

30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - loss: 0.0018
Epoch 20/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.0021
Epoch 20: loss did not improve from 0.00195
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 77ms/step - loss: 0.0021
Epoch 21/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.0019
Epoch 21: loss improved from 0.00195 to 0.00189, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 111ms/step - loss: 0.0019
Epoch 22/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - loss: 0.0018
Epoch 22: loss improved from 0.00189 to 0.00182, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_4h_model.keras
30/30 ━━━━━━━━━━━━━━━━━━━━ 4s 120ms/step - loss: 0.0018
Epoch 23/50
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0017
Epoch 23: loss improved from 0.00182 to 0.00172, saving model to C:\Users\blackrosemike\Desktop\jupyter_notebook\crypto prediction\BTCJPY_4h_model.keras
30/3

KeyboardInterrupt: 

In [2]:
from itertools import product
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import requests
import pickle
from datetime import datetime, timedelta
from celery import Celery

# Set up Celery app
# celery_app = Celery('model_training', broker='redis://localhost:6379/0')

# Define your symbols and time frames
CRYPTO_SYMBOLS = ["BTC", "ETH", "BNB", "XRP", "ADA", "SOL", "DOGE", "SHIB", " ", "TRX", "AVAX", "ATOM", "LINK"]
BASE_SYMBOLS = ["USDT", "USDC", "EUR", "JPY", "TRY"]
TIME_FRAMES = ["15m", "30m", "1h", "4h", "1d", "1w"]

# Base path for saving models and scalers
base_path = os.getcwd()

# Function to fetch candlestick data with dynamic date ranges
def fetch_candlestick_data(symbol, interval, start_time, end_time):
    BINANCE_BASE_URL = "https://api.binance.com/api/v3/klines"
    limit = 1000
    df = pd.DataFrame()

    while start_time < end_time:
        url = f"{BINANCE_BASE_URL}?symbol={symbol}&interval={interval}&limit={limit}&startTime={int(start_time.timestamp() * 1000)}"
        response = requests.get(url)
        data = response.json()

        if not data:
            break

        temp_df = pd.DataFrame(data, columns=[
            'OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume',
            'CloseTime', 'QuoteAssetVolume', 'Trades', 'TakerBuyBase', 'TakerBuyQuote', 'Ignore'])
        df = pd.concat([df, temp_df], ignore_index=True)

        start_time = datetime.fromtimestamp(temp_df['CloseTime'].iloc[-1] / 1000)

    return df[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float)

# Function to prepare data
def prepare_data(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

    x_train, y_train = [], []
    for i in range(60, len(scaled_data)):
        x_train.append(scaled_data[i-60:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    return x_train, y_train, scaler

# Function to train the model
def train_model(symbol, interval, base_path):
    model_path = os.path.join(base_path, f"{symbol}_{interval}_model.keras")
    scaler_path = os.path.join(base_path, f"{symbol}_{interval}_scaler.pkl")

    # Check if model and scaler exist
    if os.path.exists(model_path) and os.path.exists(scaler_path):
        print(f"Model and scaler already exist for {symbol} at {interval}. Skipping training.")
        return

    # Fetch data for the past 60 days
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=60)
    data = fetch_candlestick_data(symbol, interval, start_time, end_time)

    # Prepare data
    x_train, y_train, scaler = prepare_data(data)

    # Build GRU model
    model = Sequential()
    model.add(Input(shape=(x_train.shape[1], 1)))
    model.add(GRU(units=50, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    # Callbacks
    early_stopping = EarlyStopping(monitor='loss', patience=5, verbose=1)
    model_checkpoint = ModelCheckpoint(model_path, save_best_only=True, monitor='loss', verbose=1)

    # Train the model
    model.fit(x_train, y_train, batch_size=32, epochs=50, callbacks=[early_stopping, model_checkpoint])

    # Save scaler
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)

    print(f"Model and scaler saved for {symbol} at {interval}.")
    
    
# Iterate over all combinations
for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
    symbol = f"{crypto_symbol}{base_symbol}"
    print(f"Training model for {symbol} at {time_frame} interval")
    
    # Train the model
    try:
        train_model(symbol, time_frame, base_path)
    except Exception as e:
        print(f"Failed to train model for {symbol} at {time_frame}: {e}")

# Celery task to retrain models
# @celery_app.task
# def retrain_models():
#     for crypto_symbol, base_symbol, time_frame in product(CRYPTO_SYMBOLS, BASE_SYMBOLS, TIME_FRAMES):
#         symbol = f"{crypto_symbol}{base_symbol}"
#         print(f"Retraining model for {symbol} at {time_frame} interval")

#         try:
#             train_model(symbol, time_frame, base_path)
#         except Exception as e:
#             print(f"Failed to train model for {symbol} at {time_frame}: {e}")

# Entry point for manual execution
# if __name__ == "__main__":
#     retrain_models()


Training model for BTCUSDT at 15m interval
Model and scaler already exist for BTCUSDT at 15m. Skipping training.
Training model for BTCUSDT at 30m interval
Model and scaler already exist for BTCUSDT at 30m. Skipping training.
Training model for BTCUSDT at 1h interval
Model and scaler already exist for BTCUSDT at 1h. Skipping training.
Training model for BTCUSDT at 4h interval
Model and scaler already exist for BTCUSDT at 4h. Skipping training.
Training model for BTCUSDT at 1d interval
Failed to train model for BTCUSDT at 1d: tuple index out of range
Training model for BTCUSDT at 1w interval
Failed to train model for BTCUSDT at 1w: tuple index out of range
Training model for BTCUSDC at 15m interval
Model and scaler already exist for BTCUSDC at 15m. Skipping training.
Training model for BTCUSDC at 30m interval
Model and scaler already exist for BTCUSDC at 30m. Skipping training.
Training model for BTCUSDC at 1h interval
Model and scaler already exist for BTCUSDC at 1h. Skipping training.

Failed to train model for BNBEUR at 1d: tuple index out of range
Training model for BNBEUR at 1w interval
Failed to train model for BNBEUR at 1w: tuple index out of range
Training model for BNBJPY at 15m interval
Model and scaler already exist for BNBJPY at 15m. Skipping training.
Training model for BNBJPY at 30m interval
Model and scaler already exist for BNBJPY at 30m. Skipping training.
Training model for BNBJPY at 1h interval
Model and scaler already exist for BNBJPY at 1h. Skipping training.
Training model for BNBJPY at 4h interval
Model and scaler already exist for BNBJPY at 4h. Skipping training.
Training model for BNBJPY at 1d interval
Failed to train model for BNBJPY at 1d: tuple index out of range
Training model for BNBJPY at 1w interval
Failed to train model for BNBJPY at 1w: tuple index out of range
Training model for BNBTRY at 15m interval
Model and scaler already exist for BNBTRY at 15m. Skipping training.
Training model for BNBTRY at 30m interval
Model and scaler already

Failed to train model for SOLUSDT at 1d: tuple index out of range
Training model for SOLUSDT at 1w interval
Failed to train model for SOLUSDT at 1w: tuple index out of range
Training model for SOLUSDC at 15m interval
Model and scaler already exist for SOLUSDC at 15m. Skipping training.
Training model for SOLUSDC at 30m interval
Model and scaler already exist for SOLUSDC at 30m. Skipping training.
Training model for SOLUSDC at 1h interval
Model and scaler already exist for SOLUSDC at 1h. Skipping training.
Training model for SOLUSDC at 4h interval
Model and scaler already exist for SOLUSDC at 4h. Skipping training.
Training model for SOLUSDC at 1d interval
Failed to train model for SOLUSDC at 1d: tuple index out of range
Training model for SOLUSDC at 1w interval
Failed to train model for SOLUSDC at 1w: tuple index out of range
Training model for SOLEUR at 15m interval
Model and scaler already exist for SOLEUR at 15m. Skipping training.
Training model for SOLEUR at 30m interval
Model and

Failed to train model for SHIBJPY at 1d: tuple index out of range
Training model for SHIBJPY at 1w interval
Failed to train model for SHIBJPY at 1w: tuple index out of range
Training model for SHIBTRY at 15m interval
Model and scaler already exist for SHIBTRY at 15m. Skipping training.
Training model for SHIBTRY at 30m interval
Model and scaler already exist for SHIBTRY at 30m. Skipping training.
Training model for SHIBTRY at 1h interval
Model and scaler already exist for SHIBTRY at 1h. Skipping training.
Training model for SHIBTRY at 4h interval
Model and scaler already exist for SHIBTRY at 4h. Skipping training.
Training model for SHIBTRY at 1d interval
Failed to train model for SHIBTRY at 1d: tuple index out of range
Training model for SHIBTRY at 1w interval
Failed to train model for SHIBTRY at 1w: tuple index out of range
Training model for  USDT at 15m interval
Failed to train model for  USDT at 15m: single positional indexer is out-of-bounds
Training model for  USDT at 30m interv

Failed to train model for AVAXUSDT at 1d: tuple index out of range
Training model for AVAXUSDT at 1w interval
Failed to train model for AVAXUSDT at 1w: tuple index out of range
Training model for AVAXUSDC at 15m interval
Model and scaler already exist for AVAXUSDC at 15m. Skipping training.
Training model for AVAXUSDC at 30m interval
Model and scaler already exist for AVAXUSDC at 30m. Skipping training.
Training model for AVAXUSDC at 1h interval
Model and scaler already exist for AVAXUSDC at 1h. Skipping training.
Training model for AVAXUSDC at 4h interval
Model and scaler already exist for AVAXUSDC at 4h. Skipping training.
Training model for AVAXUSDC at 1d interval
Failed to train model for AVAXUSDC at 1d: tuple index out of range
Training model for AVAXUSDC at 1w interval
Failed to train model for AVAXUSDC at 1w: tuple index out of range
Training model for AVAXEUR at 15m interval
Model and scaler already exist for AVAXEUR at 15m. Skipping training.
Training model for AVAXEUR at 30m 

Failed to train model for LINKEUR at 1d: tuple index out of range
Training model for LINKEUR at 1w interval
Failed to train model for LINKEUR at 1w: tuple index out of range
Training model for LINKJPY at 15m interval
Failed to train model for LINKJPY at 15m: single positional indexer is out-of-bounds
Training model for LINKJPY at 30m interval
Failed to train model for LINKJPY at 30m: single positional indexer is out-of-bounds
Training model for LINKJPY at 1h interval
Failed to train model for LINKJPY at 1h: single positional indexer is out-of-bounds
Training model for LINKJPY at 4h interval
Failed to train model for LINKJPY at 4h: single positional indexer is out-of-bounds
Training model for LINKJPY at 1d interval
Failed to train model for LINKJPY at 1d: single positional indexer is out-of-bounds
Training model for LINKJPY at 1w interval
Failed to train model for LINKJPY at 1w: single positional indexer is out-of-bounds
Training model for LINKTRY at 15m interval
Model and scaler alread